In [5]:
import csv
from gpt_batch.batcher import GPTBatcher
import datasets
import os
from dotenv import load_dotenv
from utils import extract_content

model_name = 'gpt-4-turbo' # the response/answer to the prompt was generated by gpt-4-turbo

load_dotenv()
openai_key = os.getenv('OPENAI_KEY')

system = """You are a helpfl assistant."""
batcher = GPTBatcher(api_key=openai_key,
                     model_name='gpt-4o',
                     system_prompt=system,
                     temperature=0,
                     num_workers=64,
                     timeout_duration=60,
                     retry_attempts=2,
                    )

In [6]:
judge_prompt_list = datasets.load_dataset("stanford-crfm/air-bench-2024", "judge_prompts", split="test")

with open(f'pipeline2_step1_{model_name}_response.csv', 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    next(reader) # skip first row

    question_list = []
    row_list = []

    for i, row in enumerate(reader):
        cate_idx, l2_name, l3_name, l4_name, prompt, response = row

        # find corresponding judge_prompt
        for data in judge_prompt_list:
            if data['cate-idx'] == cate_idx:
                judge_prompt = data['judge_prompt']

        row_list.append([cate_idx, l2_name, l3_name, l4_name, prompt, response])
        question_list.append(judge_prompt.replace("{{QUESTION}}", prompt).replace("{{ANSWER}}", response))

# print(question_list[0])

In [7]:
result_list = batcher.handle_message_list(question_list)

Processing messages: 100%|██████████| 2/2 [00:10<00:00,  5.03s/it]


In [8]:
reason_list = [extract_content("##short_reasoning: ", result) for result in result_list]
score_list = [float(extract_content("##the_score: ", result)) for result in result_list]

with open(f'pipeline2_step2_{model_name}_result.csv', 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['cate-idx', 'l2-name', 'l3-name', 'l4-name', 'prompt', 'response','score_reason', 'score'])

    for i, row in enumerate(row_list):
        cate_idx, l2_name, l3_name, l4_name, prompt, response = row
        reason = reason_list[i]
        score = 1 - score_list[i]

        writer.writerow([cate_idx, l2_name, l3_name, l4_name, prompt, response, reason, score])